# 구글 드라이브 마운트
자동백업을 위해 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 패키지 설치
- Pororo
- Konlpy

In [8]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 33.6 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [9]:
!git clone https://github.com/kakaobrain/pororo.git

Cloning into 'pororo'...
remote: Enumerating objects: 2119, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2119 (delta 11), reused 22 (delta 8), pack-reused 2090
Receiving objects: 100% (2119/2119), 13.09 MiB | 12.66 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.


In [10]:
cd pororo

/content/pororo


In [11]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/pororo
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.8/748.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (set

In [12]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 kB 39.0 MB/s eta 0:00:00


# 데이터 전처리
## 아이디어
- re 사용하여 데이터 정제
- 데이터 살펴보고 예외처리
- 토크나이저로 명사만 가져오기
- Pororo 제로샷학습 상관관계 분석 후 최소값 제거(5단어 이상일 경우만)

In [13]:
# Hannanum(한나눔)
from konlpy.tag import Hannanum
tokenizer=Hannanum()

In [14]:
from pororo import Pororo
from tqdm import tqdm 
import numpy as np
import re

## 사용한 함수들

In [36]:
## 원시데이터 정제
def first_eda(df):
  df['title'] = [re.sub('[0-9]+[개|팩|인]', '', s) for s in df['title']]
  df['title'] = [re.sub('[()]', '', s) for s in df['title']]
  df['title'] = [re.sub('[^가-힣\s]', '', s) for s in df['title']]
  df['title'] = [re.sub('주문', '', s) for s in df['title']]
  df['title'] = [re.sub('[\xa0]', ' ', s) for s in df['title']]
  df['title'] = [re.sub('전라도', '', s) for s in df['title']]

  df.drop_duplicates(['title'], inplace=True)
  df.drop(df.index[3], inplace=True)
  df.dropna(axis=0, inplace=True)
  df.reset_index(drop=True, inplace=True)
  return df

In [59]:
## 토큰화(토크나이저 사용)
def tokenizing(titles):
  title = []
  token_list = []
  for text in titles:
    tt = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", text)
    ttt = tt.replace(u'\xa0', u'')
    ttt = tt.replace(u'\u3000', u'')
    if ttt[0] == ' ':
      title.append('쓰레기')
    else:
      title.append(ttt)
  for t in tqdm(title):
    token_list.append(tokenizer.nouns(t))
  return token_list

In [16]:
## 토큰 중복 제거
def dupcheck(title_token):
  title = []
  for i in tqdm(title_token):
    result = []
    for j in i:
      if j not in result:
        result.append(j)
    title.append(result)
  return title

In [131]:
## 토큰 전처리
def token_eda(category, title_token):
  for i in tqdm(range(len(category))): 
    if len(title_token[i]) >=6:
      result = topic_cls(category[i], title_token[i])
      trash = min(result, key=(lambda k: result[k]))
      title_token[i].remove(trash)
  return title_token

In [17]:
## 토큰들 문자열로 결합
def tokenToString(token_list):
    res = []
    for token in tqdm(token_list):
      s = ' '.join(token)
      res.append(s)
    return res

In [18]:
## 쓰레기 데이터 제거(토크나이징EDA 산물)
def trash_check(df):
  idx = df[df['eda_title'] == '쓰레기'].index
  df.drop(idx, inplace=True)
  df.dropna(axis=0, inplace=True)
  df.reset_index(drop=True, inplace=True)
  return df

# 전처리 (1차 시도)

In [2]:
import pandas as pd

In [39]:
data = pd.read_csv('/content/final_ver1.csv', encoding='UTF-8', index_col=0)

In [ ]:
# 데이터 준비
df = first_eda(data)

In [46]:
df

,title,category,sub_category,price
0,국내산 배추 김치 포기 김장김치 주문,김치,포기김치,15900
1,피코크 조선호텔 포기김치,김치,포기김치,51040
2,대상 종가집 행복이온 포기김치,김치,포기김치,34200
3,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치,김치,포기김치,11080
4,엄마손맛 포기김치,김치,포기김치,27900
...,...,...,...,...
241851,먹템 매콤한 낙곱새 극락 밀키트인분 팩 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370
241852,연말축제 청정원 호밍스 한우진곰탕 개,밀키트,찌개/국,25500
241853,프레시지 프레시지 간편한 직화용기 꼬치어묵탕,밀키트,찌개/국,11860
241854,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스 인분,밀키트,찌개/국,16900


In [ ]:
# 토크나이징
title_token = tokenizing(df['title'])
titles = dupcheck(title_token)

In [22]:
# Pororo 제로샷 분류모델
topic_cls = Pororo(task='zero-topic', lang='ko')

In [ ]:
# 토큰 EDA / 데이터저장
titl_list = token_eda(df['sub_category'], titles)
new = tokenToString(titl_list)
eda = pd.DataFrame(zip(new))
df['eda_title'] = eda[0]
df.to_csv('/content/drive/MyDrive/project3/finish.csv', index=False)

# 결론


---


# 토크나이징의 한계
1. 명사 분류의 문제 - (소고기 -> 소고, 오리 -> 오)
-기, -리 등을 무조건 조사 처리하는 문제점.
2. 토크나이저마다 예외 처리 방식이 다름.

Okt의 경우 별다른 예외처리가 필요없었지만,

Komoran, Hannanum은 null값 처리가 필요했다.

명사 토큰화에서 Hannanum이 상대적으로 명사를 조사와 분류하지 않는 경향성을 보여 선택했지만 일부였다.



---



## 따라서

전처리에 토크나이저를 사용하지 않고

split으로 단어를 나눠 처리한 뒤 재결합 하는 방법을 채택.

.isdigit(), .isalnum() 을 사용해 예외를 처리.

- any(), isdigit() - 한글자라도 숫자가 같이 묶인 단어는 포함X (1개, 1통, 1병, 1팩 등의 단위)

- .isalnum() - 글자 단위로 나눠 특수문자만 제거하고 가져오기

- re.sub() 로 영어 제거, 공백은 split()후 재결합으로 제거.

- 품명에 한 단어만 존재하는 경우 제거 (브랜드명만 있는 경우가 있어 의미 없다 판단함)



# 전처리 (2차 시도)
## 토크나이저 사용 X

In [165]:
data2 = pd.read_csv('/content/final.csv', encoding='UTF-8', index_col=0)

In [166]:
data2.reset_index(drop=True, inplace=True)

In [167]:
data2

,title,category,sub_category,price
0,국내산 100% 전라도 배추 김치 포기 김장김치 주문,김치,포기김치,15900
1,피코크 조선호텔 포기김치 8kg,김치,포기김치,51040
2,대상 종가집 행복이온 포기김치 10kg,김치,포기김치,34200
3,대상 종가집 전라도 행복이온 포기김치 10kg,김치,포기김치,46840
4,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치 800g,김치,포기김치,11080
...,...,...,...,...
265030,먹템 매콤한 낙곱새 극락 밀키트(2-3인분) 4팩 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370
265031,[연말축제 1+1] 청정원 호밍스 한우진곰탕 450g X 10개,밀키트,찌개/국,25500
265032,프레시지 프레시지 간편한 직화용기 꼬치어묵탕 818g,밀키트,찌개/국,11860
265033,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스 (2인분),밀키트,찌개/국,16900


In [168]:
## split(). 공백 기준 단어 분리
token_list = []
for title in data2['title']:
  res = title.split()
  token_list.append(res)

In [169]:
## 중복 단어 제거
token_list = dupcheck(token_list)

100%|██████████| 265035/265035 [00:00<00:00, 398928.34it/s]


In [170]:
## 전처리
tok = []
for text in tqdm(token_list):
  li = []
  for i in text:
    if any(temp.isdigit() for temp in i) == False: # 숫자와 붙어서 단어로 묶인 것들은 포함하지 않는다.
      res = ""
      for word in i:
        if word.isalnum(): # 단어별로 한글만을 가져온다(특수문자 포함X)
          res+=word
          i = res
      li.append(i)
  tok.append(li)

100%|██████████| 265035/265035 [00:02<00:00, 93583.14it/s]


In [171]:
## 중복 토큰 제거
tok2 = dupcheck(tok)

100%|██████████| 265035/265035 [00:00<00:00, 729165.82it/s]


In [172]:
## 토큰 재결합하여 문자열로
tok3 = tokenToString(tok2)

100%|██████████| 265035/265035 [00:00<00:00, 1398748.72it/s]


In [173]:
## 토큰화
def text(titles):
  title = []
  for text in titles:
    tt = re.sub(r"[^가-힣\s]", "", text) # 영어 제외
    ttt = tt.replace(u'\xa0', u'') # \xa0, \u3000 제외
    ttt = tt.replace(u'\u3000', u'')
    if ttt == '':   # 모든 제품명이 영어인 경우
      ttt = '쓰레기'  # 한글 단어로 할당해주고 나중에 한번에 제거
    title.append(ttt)
  return title

In [174]:
tok4 = text(tok3)

In [175]:
tokens = []
for i in tok4:
  res = i.split() # split() 공백기준으로 나눠주어 공백 제거
  tokens.append(res)

In [176]:
## 한 단어만 있는 데이터 갯수('쓰레기' 포함)
count=0
for i in tokens:
  if len(i) == 1:
    count+=1
count

13108

In [177]:
results = tokenToString(tokens)
data2['new']=results
data2

100%|██████████| 265035/265035 [00:00<00:00, 1582374.91it/s]


,title,category,sub_category,price,new
0,국내산 100% 전라도 배추 김치 포기 김장김치 주문,김치,포기김치,15900,국내산 전라도 배추 김치 포기 김장김치 주문
1,피코크 조선호텔 포기김치 8kg,김치,포기김치,51040,피코크 조선호텔 포기김치
2,대상 종가집 행복이온 포기김치 10kg,김치,포기김치,34200,대상 종가집 행복이온 포기김치
3,대상 종가집 전라도 행복이온 포기김치 10kg,김치,포기김치,46840,대상 종가집 전라도 행복이온 포기김치
4,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치 800g,김치,포기김치,11080,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치
...,...,...,...,...,...
265030,먹템 매콤한 낙곱새 극락 밀키트(2-3인분) 4팩 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370,먹템 매콤한 낙곱새 극락 캠핑음식 용밀 간편조리식품 키트
265031,[연말축제 1+1] 청정원 호밍스 한우진곰탕 450g X 10개,밀키트,찌개/국,25500,연말축제 청정원 호밍스 한우진곰탕
265032,프레시지 프레시지 간편한 직화용기 꼬치어묵탕 818g,밀키트,찌개/국,11860,프레시지 간편한 직화용기 꼬치어묵탕
265033,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스 (2인분),밀키트,찌개/국,16900,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스


In [ ]:
# data2.loc[173282]

In [180]:
## 한단어만 있는 경우와 '쓰레기' 데이터 행 제거
idx = -1
for data in tqdm(data2['new']):
  idx += 1
  if len(data.split()) == 1:  
    data2 = data2.drop([idx], axis = 0)
data2.info()

100%|██████████| 265035/265035 [07:24<00:00, 596.17it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251927 entries, 0 to 265034
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         251927 non-null  object
 1   category      251927 non-null  object
 2   sub_category  251927 non-null  object
 3   price         251927 non-null  int64 
 4   new           251927 non-null  object
dtypes: int64(1), object(4)
memory usage: 11.5+ MB


In [181]:
data2[data2['new'] == '쓰레기'] # 모두 지워진 것을 확인

,title,category,sub_category,price,new


In [182]:
df = data2[['new', 'category', 'sub_category', 'price']]
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'new':'title'})
df

,title,category,sub_category,price
0,국내산 전라도 배추 김치 포기 김장김치 주문,김치,포기김치,15900
1,피코크 조선호텔 포기김치,김치,포기김치,51040
2,대상 종가집 행복이온 포기김치,김치,포기김치,34200
3,대상 종가집 전라도 행복이온 포기김치,김치,포기김치,46840
4,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치,김치,포기김치,11080
...,...,...,...,...
251922,먹템 매콤한 낙곱새 극락 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370
251923,연말축제 청정원 호밍스 한우진곰탕,밀키트,찌개/국,25500
251924,프레시지 간편한 직화용기 꼬치어묵탕,밀키트,찌개/국,11860
251925,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스,밀키트,찌개/국,16900


In [128]:
## 여기 까지 eda한 데이터 저장
df.to_csv('/content/drive/MyDrive/project3/super_final_ver1.csv', index=False)

In [ ]:
## 토큰화 해주고 (Pororo 돌리기 위해)
token = []
for title in df['title']:
  res = title.split()
  token.append(res)

In [184]:
## 카테고리와 서브카테고리를 합친 문장으로 Pororo 분류
cats = df['category']+" "+df['sub_category']
cat = cats.str.replace("/"," ")
cat

0          김치 포기김치
1          김치 포기김치
2          김치 포기김치
3          김치 포기김치
4          김치 포기김치
            ...   
251922    밀키트 찌개 국
251923    밀키트 찌개 국
251924    밀키트 찌개 국
251925    밀키트 찌개 국
251926    밀키트 찌개 국
Length: 251927, dtype: object

In [186]:
# 토큰 EDA / 데이터저장
titl_list = token_eda(cat, token)
new = tokenToString(titl_list)
eda = pd.DataFrame(zip(new))
df['eda_title_2'] = eda[0]
df.to_csv('/content/drive/MyDrive/project3/finish_retry.csv', index=False)

100%|██████████| 251927/251927 [00:00<00:00, 1567040.32it/s]


In [187]:
df

,title,category,sub_category,price,eda_title_2
0,국내산 전라도 배추 김치 포기 김장김치 주문,김치,포기김치,15900,국내산 배추 김치 포기 김장김치 주문
1,피코크 조선호텔 포기김치,김치,포기김치,51040,피코크 조선호텔 포기김치
2,대상 종가집 행복이온 포기김치,김치,포기김치,34200,대상 종가집 행복이온 포기김치
3,대상 종가집 전라도 행복이온 포기김치,김치,포기김치,46840,대상 종가집 전라도 행복이온 포기김치
4,선화동매운실비김치 맛있게 매운 대전 선화동 실비 김치,김치,포기김치,11080,선화동매운실비김치 맛있게 매운 선화동 실비 김치
...,...,...,...,...,...
251922,먹템 매콤한 낙곱새 극락 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370,먹템 매콤한 극락 캠핑음식 용밀 간편조리식품 키트
251923,연말축제 청정원 호밍스 한우진곰탕,밀키트,찌개/국,25500,연말축제 청정원 호밍스 한우진곰탕
251924,프레시지 간편한 직화용기 꼬치어묵탕,밀키트,찌개/국,11860,프레시지 간편한 직화용기 꼬치어묵탕
251925,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스,밀키트,찌개/국,16900,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스


In [188]:
df2 = df[['eda_title_2', 'category', 'sub_category', 'price']]
df2

,eda_title_2,category,sub_category,price
0,국내산 배추 김치 포기 김장김치 주문,김치,포기김치,15900
1,피코크 조선호텔 포기김치,김치,포기김치,51040
2,대상 종가집 행복이온 포기김치,김치,포기김치,34200
3,대상 종가집 전라도 행복이온 포기김치,김치,포기김치,46840
4,선화동매운실비김치 맛있게 매운 선화동 실비 김치,김치,포기김치,11080
...,...,...,...,...
251922,먹템 매콤한 극락 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370
251923,연말축제 청정원 호밍스 한우진곰탕,밀키트,찌개/국,25500
251924,프레시지 간편한 직화용기 꼬치어묵탕,밀키트,찌개/국,11860
251925,아임셰프 마라 밀푀유나베 밀키트 쿠킹박스,밀키트,찌개/국,16900


In [189]:
df2['category'].unique()

array(['김치', '반찬', '수산물', '장류', '농산물', '축산물', '건강식품', '통조림/캔', '음료',
       '냉동/간편조리식품', '조미료', '라면/면류', '소스/드레싱', '과자/베이커리', '오일/가루/향신료',
       '다이어트식품', '밀키트', '잼/시럽', '유가공품', '전통주', '커피/차류', '간편조리식품',
       '과자/떡/베이커리', '생수/음료'], dtype=object)

## 카테고리 조정 및 저장

1. 음료 + 생수/음료 >> 생수/음료
2. 과자/베이커리 + 과자/떡/베이커리 >> 과자/떡/베이커리
3. 냉동/간편조리식품 + 간편조리식품 >> 냉동/간편조리식품

In [190]:
def jojung(df, before, after):
  for idx, value in enumerate(before):
    df.loc[df['category'] == value,'category'] = after[idx] 
  return df

In [ ]:
before = ['음료', '과자/베이커리', '간편조리식품']
after = ['생수/음료', '과자/떡/베이커리', '냉동/간편조리식품']
final = jojung(df2, before, after)

In [199]:
final = final.rename(columns={'eda_title_2':'title'})

In [193]:
final['category'].unique()

array(['김치', '반찬', '수산물', '장류', '농산물', '축산물', '건강식품', '통조림/캔', '생수/음료',
       '냉동/간편조리식품', '조미료', '라면/면류', '소스/드레싱', '과자/떡/베이커리', '오일/가루/향신료',
       '다이어트식품', '밀키트', '잼/시럽', '유가공품', '전통주', '커피/차류'], dtype=object)

In [200]:
final.isnull().sum()

title           0
category        0
sub_category    0
price           0
dtype: int64

In [201]:
final.drop_duplicates(['title'], inplace=True)
final.reset_index(drop=True, inplace=True)
final

,title,category,sub_category,price
0,국내산 배추 김치 포기 김장김치 주문,김치,포기김치,15900
1,피코크 조선호텔 포기김치,김치,포기김치,51040
2,대상 종가집 행복이온 포기김치,김치,포기김치,34200
3,대상 종가집 전라도 행복이온 포기김치,김치,포기김치,46840
4,선화동매운실비김치 맛있게 매운 선화동 실비 김치,김치,포기김치,11080
...,...,...,...,...
205375,청정원 호밍스 사골 시래기 된장국,밀키트,찌개/국,30400
205376,먹템 매콤한 극락 캠핑음식 용밀 간편조리식품 키트,밀키트,찌개/국,63370
205377,연말축제 청정원 호밍스 한우진곰탕,밀키트,찌개/국,25500
205378,프레시지 간편한 직화용기 꼬치어묵탕,밀키트,찌개/국,11860


In [202]:
final.duplicated().sum()

0

In [203]:
final.to_csv('/content/drive/MyDrive/project3/real_finish.csv', index=False)

In [204]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205380 entries, 0 to 205379
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         205380 non-null  object
 1   category      205380 non-null  object
 2   sub_category  205380 non-null  object
 3   price         205380 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 6.3+ MB
